# Datan analysointi K2020_2 - Tehtävät kerta 4

Palauta Optiman palautuslaatikkoon viimeistään su 1.3. Kirjoita palautuslaatikon kommenttikenttään, mitkä tehtävät olet tehnyt kokonaan/osittain.

**Opiskelijan nimi:** Jere Pesonen

## 1. Yhdistäminen

Tiedostossa https://student.labranet.jamk.fi/~varpe/datananal2k2020/kerta4/pyorat.txt on kesältä 2017 tietoja vapaana olevien kaupunkipyörien määristä neljästä telineestä Helsingistä (keskiarvot tunneittain).

Tiedostossa https://student.labranet.jamk.fi/~varpe/datananal2k2020/kerta4/helsinki2017.csv on vastaavalta ajalta säähavaintoja Helsingin Kaisanimestä. 

Lue molemmat tiedosto omiin DataFrameihinsa ja yhdistä DataFramet järkevästi, siten että molempien datojen vastaavat tunnit on samalla rivillä.


Tee yhdistäminen niin, että kaikki tunnit jotka on jommassa kummassa tiedostossa, on mukana yhdistetyssä DataFramessa (pyörädatasta puuttuu tunteja välistä).


Poista mahdolliset ylimääräiset sarakkeet siten, että kk/pv/tunti -tiedot eivät ole kahdesti.





In [469]:
import pandas as pd 
import numpy as np

df = pd.read_csv("https://student.labranet.jamk.fi/~varpe/datananal2k2020/kerta4/pyorat.txt")
df2 = pd.read_csv("https://student.labranet.jamk.fi/~varpe/datananal2k2020/kerta4/helsinki2017.csv")

df2.drop(['rivi'], axis=1, inplace=True)
df2['Klo'] = df2['Klo'].str.split(':').str.get(0).astype(int)
df.rename(columns={"Month":"Kk", "Day":"Pv", "Hour":"Klo"}, inplace=True)
#print(df.head(50))
#print( "-"*70)
#print(df2.head(50))
#print("-"*70)

df3 = pd.merge(df2, df, on = ['Kk', 'Pv', 'Klo'], how='outer')
print(df3.head(50))

    Kk  Pv  Klo  Pilvien määrä (1/8)  Suhteellinen kosteus (%)  \
0    6   1    0                    1                        61   
1    6   1    1                    1                        68   
2    6   1    2                    5                        72   
3    6   1    3                    5                        74   
4    6   1    4                    7                        72   
5    6   1    5                    1                        66   
6    6   1    6                    0                        59   
7    6   1    7                    3                        50   
8    6   1    8                    5                        81   
9    6   1    9                    7                        83   
10   6   1   10                    5                        75   
11   6   1   11                    5                        75   
12   6   1   12                    7                        81   
13   6   1   13                    1                        65   
14   6   1

## 2. Ristiintaulukointia
Tiedostossa https://student.labranet.jamk.fi/~varpe/datananal2k2020/kerta4/kysely.csv on kuvitteellisen henkilöstökyselyn tuloksia.

Dataan on koodattu koulutus-kenttään numeroilla:  
* 1 = peruskoulu
* 2 = toisen asteen koulutus
* 3 = korkeakoulu
* 4 = ylempi korkeakoulu

TyytJohto ja TyytTyöteht -kentät kuvaavat tyytyväisyyttä johtoon ja työtehtäviin asteikolla 1-5.   

työterveys, liikunta ja lomaosake-kentät sisältävät ykkösen jos vastaaja on käyttänyt näitä palveluita, muuten NaN

Ristiintaulukoi/pivotoi/koosta seuraavat jutut:  
* lukumäärät siten, että riveillä perhesuhteet, sarakkeissa tyytyväisyys johtoon (lukumäärät, eli kuinka monta perheellistä on vastannut "tyytyväisyys johtoon"-kohtaan ykkösen jne.
  
  
* prosenttiosuudet siten, että riveillä koulutus tekstinä (esim peruskoulu), sarakkeissa tyytyväisyys työtehtäviin ( prosenttiosuudet 1 desimaalilla riveittäin eli esim montako % vain peruskoulun käyneistä vastaa 1 jne.) 
  
  
* keskiarvot tyytyväisyyksistä johtoon ja työtehtäviin siten, että riveillä on ikäluokka 20-29, 30-39, ... 60-69 ja (eri sarakkeissa) tyytyväisyydet johtoon ja työtehtäviin  
  
  
* kuinka monta prosenttia miehistä ja naisista on käyttäneet työterveys, liikunta ja lomaosake-palveluita  
malli tuloksesta
```
          liikunta lomaosake työterveys
sukupuoli                              
mies         24.6%     11.5%      57.4%
nainen       23.8%      9.5%      57.1%
```

In [251]:
df = pd.read_csv("https://student.labranet.jamk.fi/~varpe/datananal2k2020/kerta4/kysely.csv")
df2 = pd.DataFrame()
df2['perhetilanne'] = df['perhetilanne']
df2['TyytJohto'] = df['TyytJohto']
df2 = df2.groupby(['perhetilanne', 'TyytJohto']).size()
df2 = df2.unstack()
print(df2)
print('-'*50)

df3 = pd.DataFrame()
df3['koulutus'] = df['koulutus']
df3['TyytTyöteht'] = df['TyytTyöteht']
df3 = df3.groupby(['koulutus', 'TyytTyöteht']).size()
df3 = df3.unstack()
df3 = df3.reset_index()
df3[[1, 2, 3, 4, 5]] = df3[[1, 2, 3, 4, 5]].apply(lambda x: x/x.sum(), axis=1)
df3['koulutus'] = df3['koulutus'].map({1.0:"peruskoulu", 2.0:"toisen asteen koulutus", 3.0:"korkeakoulu", 4.0:"ylempi korkeakoulu"})
df3[1] = ((df3[1] * 100).round(1).astype(str) + " %")
df3[2] = ((df3[2] * 100).round(1).astype(str) + " %")
df3[3] = ((df3[3] * 100).round(1).astype(str) + " %")
df3[4] = ((df3[4] * 100).round(1).astype(str) + " %")
df3[5] = ((df3[5] * 100).round(1).astype(str) + " %")
df3 = df3.set_index('koulutus')
print(df3)
print('-'*50)
#min ikä 20, max ikä 61
df4 = pd.DataFrame()
rajat = [20,25,30,35,40,45,50,55,60,65]
arvosanat = ['20-25','25-30','30-35','35-40','40-45','45-50','50-55','55-60','60-65']
df['ikäluokka'] = pd.cut(df['ikä'],rajat, labels=arvosanat)
df4['KeskiTyytTyöteht'] = df['TyytTyöteht'].groupby(df['ikäluokka']).mean().round(1)
df4['KeskiTyytJohto'] = df['TyytJohto'].groupby(df['ikäluokka']).mean().round(1)
print(df4)
print('-'*50)
#print(df.head(10))
#print('-'*50)
#df5 = df.groupby(['sukupuoli', 'työterveys']).sum()
df5 = pd.DataFrame(df['liikunta'].groupby(df['sukupuoli']).count())
df5['lomaosake'] = df['lomaosake'].groupby(df['sukupuoli']).count()
df5['työterveys'] = df['työterveys'].groupby(df['sukupuoli']).count()
df5['lukumäärä'] = df['sukupuoli'].value_counts()
df5[['liikunta', 'työterveys', 'lomaosake']] = df5[['liikunta', 'työterveys', 'lomaosake']].apply(lambda x: ((x/df5['lukumäärä'])*100).round(1).astype(str) + " %")
df5.drop(columns = 'lukumäärä', inplace = True)
print(df5)

TyytJohto     1  2   3   4  5
perhetilanne                 
perheellinen  3  9  17  17  5
perheetön     4  7  13   6  1
--------------------------------------------------
TyytTyöteht                  1       2       3       4       5
koulutus                                                      
peruskoulu              11.1 %  11.1 %  48.1 %  22.2 %   7.4 %
toisen asteen koulutus   3.3 %  26.7 %  33.3 %  23.3 %  13.3 %
korkeakoulu              4.5 %   9.1 %  27.3 %  54.5 %   4.5 %
ylempi korkeakoulu       nan %  50.0 %   nan %   nan %  50.0 %
--------------------------------------------------
           KeskiTyytTyöteht  KeskiTyytJohto
ikäluokka                                  
20-25                   2.5             3.2
25-30                   3.4             2.9
30-35                   3.2             2.9
35-40                   2.9             2.9
40-45                   3.3             3.2
45-50                   3.6             3.0
50-55                   3.2             3.5
55-6

## 3. Liikennelaskentaa

Tiedostossa
http://student.labranet.jamk.fi/~varpe/datananal2k2020/kerta4/lam.csv  
on automaattisen liikennelaskennan tietoja yhden kuukauden ajalta yhdestä Espoon mittauspisteestä. 

* rivinro
* päiväys
* suuntakoodi (1 tai 2) 
* ajoneuvoluokka
* tuntiväli
* määrä

Koosta/muokkaa taulukosta DataFrame, jossa on 24 riviä, yksi vuorokauden jokaiselle tunnille ja 30 saraketta kuukauden päiville, siten että DataFramessa on kyseisen tunnin ajoneuvomäärien summa (molemmat suunnat, kaikki ajoneuvoluokat). 

Mallia (otsikot voi vaihdella):  
![lam1](https://student.labranet.jamk.fi/~varpe/datananal2k2020/kerta4/malli1.png)

Laske sitten loppuun sarakkeet:
* suurin, joka kertoo kunkin tunnin suurimman ajoneuvomäärän 30 päivän joukosta
* sija, joka kertoo päivän 1 (1.11.2019) sijoituksen ko. tunnin ajoneuvomäärien suhteen 30 päivän joukossa (eniten = 1.)  
    (siis jos 00-01 -tunneista 1.11. on enemmän kuin minään muuna päivänä, sija = 1. Jos vähemmän kuin muina päivinä, sija = 30)

![lam2](https://student.labranet.jamk.fi/~varpe/datananal2k2020/kerta4/malli2.png)

In [646]:
df = pd.read_csv("http://student.labranet.jamk.fi/~varpe/datananal2k2020/kerta4/lam.csv")
df2 = pd.DataFrame()
df2['Päivä'] = df['Pvm'].str.split('.').str.get(0).astype(int)
df2['tuntiväli'] = df['tuntiväli']
df2['määrä'] = df['määrä']
df2 = df2['määrä'].groupby([df2['tuntiväli'], df2['Päivä']]).sum()
df2 = df2.unstack()
df3 = df2.rank(1, ascending=False)
df2['suurin'] = df2.max(axis=1)
df2['sija'] = (df3[1])
df2.head(10)

Päivä,1,2,3,4,5,6,7,8,9,10,...,23,24,25,26,27,28,29,30,suurin,sija
tuntiväli,,,,,,,,,,,,,,,,,,,,,
00-01,183.0,278.0,321.0,140.0,156.0,160.0,172.0,185.0,361.0,319.0,...,363.0,408.0,165.0,164.0,193.0,168.0,191.0,403.0,408.0,14.0
01-02,124.0,169.0,265.0,135.0,104.0,96.0,102.0,97.0,178.0,170.0,...,199.0,254.0,105.0,134.0,115.0,130.0,114.0,245.0,265.0,14.0
02-03,84.0,145.0,147.0,53.0,78.0,80.0,69.0,104.0,129.0,127.0,...,152.0,134.0,82.0,97.0,82.0,84.0,90.0,158.0,158.0,16.5
03-04,108.0,118.0,109.0,87.0,86.0,83.0,91.0,101.0,106.0,85.0,...,126.0,123.0,148.0,95.0,88.0,102.0,96.0,135.0,148.0,9.0
04-05,217.0,157.0,128.0,229.0,204.0,219.0,203.0,221.0,140.0,118.0,...,134.0,125.0,248.0,227.0,216.0,230.0,217.0,152.0,248.0,12.0
05-06,914.0,267.0,171.0,987.0,955.0,1011.0,925.0,899.0,290.0,159.0,...,273.0,163.0,964.0,972.0,962.0,948.0,916.0,293.0,1011.0,17.5
06-07,2541.0,388.0,259.0,2987.0,2920.0,2825.0,2802.0,2435.0,372.0,192.0,...,436.0,197.0,2973.0,2859.0,2872.0,2869.0,2520.0,382.0,3026.0,18.0
07-08,3160.0,509.0,334.0,3555.0,3601.0,3615.0,3486.0,2971.0,507.0,222.0,...,583.0,257.0,3504.0,3439.0,3489.0,3425.0,2935.0,588.0,3615.0,17.0
08-09,2601.0,875.0,520.0,3075.0,3107.0,3026.0,3176.0,2605.0,865.0,338.0,...,927.0,446.0,2989.0,3057.0,3023.0,2859.0,2577.0,861.0,3223.0,19.0


## 4. Isomman tiedoston lukemien ja analysointi
Testaile isomman tiedoston lukemista tiedostolla (200 Mt):  
http://gpspekka.kapsi.fi/trafi56.zip

Kannattanee tallentaa tiedosto ensin omalle koneelle, jotta testailuvaiheessa ei tarvitse ladata sitä verkon yli joka kerta. Tiedostoa ei tarvitse purkaa.

Tiedostossa on n. 5 miljoonaa riviä "kaikkien liikennekäytössä olevien ajoneuvojen rekisteröinti-, hyväksyntä- ja teknisiä tietoja Trafin ylläpitämästä ajoneuvoliikennerekisteristä."

Data on puolipisteellä eroteltua, ja lukeminen vaatii (ilmeisesti) read_csv:lle parametrin `encoding='latin_1'`.

Aloita testailu antamalla read_csv:lle parametriksi vaikka `nrows=1000` jolloin se lukee vain 1000 ensimmäistä riviä, joilla voit tutkia esim. sarakeotsikoita. Kokeile sitten kuinka paljon rivejä saat mielekkäästi luettua kasvattamalla nrows-arvoa. Muista tallentaa tiedosto ennen kokeiluja, jotta systeemin kaatumien ei vie aikaisempia tehtäviäsi.

Kellota lukuoperaatio lisäämällä lukevan solun 1. riviksi `%%time`.

Kun olet saanut luettua "paljon"/kaikki dataa, tee sitten uuteen soluun jotain hakuja/koonteja/ryhmittelyjä (siksi uuteen soluun, että koodin ajaminen ei lue tiedostoa joka kerta uudestaan.)

Selvitä vaikka:  
* kuinka monta (vielä liikennekäytössä eli tiedostossa olevaa) ajoneuvoa käyttöönotettiin (kayttoonottopvm) 18-vuotissyntymäpäivänäsi  


* mikä on viiden kärki kentän merkkiSelvakielinen lukumäärissä  


* tee uusi sarake rekvuosi, johon luet 4 ensimmäistä merkkiä kentästä ensirekisterointipvm. Ristiintaulukoi sitten kentän sahkohybridi prosenttiosuudet (True/False) rekvuoden mukaan vuodesta 2001 eteenpäin.

In [650]:
%%time
df = pd.read_csv(r"file:///C:/Users/JereP/Documents/trafi56.zip", sep=';', decimal=',', encoding='latin_1', nrows=100000)
df

Wall time: 626 ms


,ajoneuvoluokka,ensirekisterointipvm,ajoneuvoryhma,ajoneuvonkaytto,variantti,versio,kayttoonottopvm,vari,ovienLukumaara,korityyppi,...,vaihteidenLkm,kaupallinenNimi,voimanvalJaTehostamistapa,tyyppihyvaksyntanro,yksittaisKayttovoima,kunta,Co2,matkamittarilukema,valmistenumero2,jarnro
0,MUU,NaN,21.0,1.0,NaN,NaN,19670000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,91,NaN,NaN,NaN,1
1,MUU,1976-09-01,13.0,1.0,NaN,NaN,19760000,NaN,NaN,NaN,...,NaN,NaN,1.0,NaN,NaN,893,NaN,NaN,NaN,2
2,M1,1984-07-09,NaN,1.0,NaN,NaN,19840000,1,NaN,NaN,...,NaN,FIESTA,5.0,NaN,1.0,564,NaN,NaN,NaN,3
3,MUU,1983-09-22,13.0,1.0,NaN,NaN,19830000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,734,NaN,NaN,NaN,4
4,O1,1994-02-09,1.0,1.0,NaN,NaN,19940209,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,761,NaN,NaN,NaN,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,M1,2005-09-16,NaN,1.0,AAYMF1,FM6EA0R92P,20011122,6,NaN,NaN,...,NaN,NaN,5.0,e1*98/14*0051*18,2.0,92,186.0,435921.0,WAUZZZ4B32,99996
99996,M1,2005-12-21,29.0,1.0,NaN,NaN,19940914,9,NaN,NaN,...,NaN,NaN,5.0,NaN,2.0,167,NaN,235464.0,VSA6313341,99997
99997,M1,2005-11-15,NaN,1.0,AZT250(W),AZT250L-AWPGHW(1E),20051115,0,NaN,AC,...,NaN,AVENSIS,5.0,e11*01/116*0196*04,1.0,905,224.0,165161.0,SB1EJ56L70,99998
99998,MUU,1990-09-11,509.0,1.0,NaN,NaN,19900911,NaN,NaN,NaN,...,NaN,NaN,2.0,NaN,2.0,280,NaN,NaN,NaN,99999


In [513]:
df = pd.read_csv(r"file:///C:/Users/JereP/Documents/trafi56.zip", sep=';', decimal=',', encoding='latin_1')
df.fillna(0,inplace=True)
df['rekvuosi'] = df['ensirekisterointipvm'].str.split('-').str.get(0)
#df['kayttoonottopvm'].astype(str).value_counts()['20132707'] # 0 kappaletta
df2 = pd.DataFrame(df['merkkiSelvakielinen'].value_counts(ascending=False))
df2.head(5)

,merkkiSelvakielinen
Toyota,431928
Ford,280639
Volkswagen,269289
Volvo,261573
Mercedes-Benz,233640


In [517]:
df = df[df['rekvuosi'].astype(int) > 2000]
pd.crosstab(df['sahkohybridi'], df['rekvuosi'], normalize = 'columns').applymap("{:.1%}".format)
#df.head(10)

rekvuosi,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
sahkohybridi,,,,,,,,,,,,,,,,,,,
0,100.0%,100.0%,100.0%,100.0%,100.0%,100.0%,100.0%,100.0%,100.0%,99.8%,99.6%,99.3%,98.7%,98.6%,98.2%,97.0%,94.4%,91.3%,88.7%
True,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.2%,0.4%,0.7%,1.3%,1.4%,1.8%,3.0%,5.6%,8.7%,11.3%


## 5. Jalkapalloa
Tiedostossa https://student.labranet.jamk.fi/~varpe/datananal2k2020/kerta4/epl4.txt  on Englannin Valioliigan ottelutulokset kauden 2018-2019 loppuun. Käsittelyn helpottamiseksi siinä on laskettu valmiiksi koti- ja vierasjoukkueiden saamat pisteet ottelusta (3, 1 tai 0).

Tiedostossa https://student.labranet.jamk.fi/~varpe/datananal2k2020/kerta4/sijat.txt on joukkueiden loppusijoitukset kausittain. 

Ensimmäisestä tiedostosta pitäisi saada koostettua DataFrame, jossa on tieto **kuinka monta pistettä kukin joukkue on saanut kunkin kauden 15 ensimmäisestä ottelustaan**

Toista tiedostoa hyväksi käyttäen pitäisi sitten etsiä vastaus kysymykseen **mitkä ovat olleet suurimmat erot (eniten jäljessä) neljänteen sijaan 15 ottelun jälkeen joukkueilla, jotka ovat sijoittuneet kauden lopussa neljän parhaan joukkoon?**

* laske jokaiselle joukkueelle jokaiselle kaudelle kuinka paljon pisteitä on kauden 15 ensimmäisestä pelistä
* laske jokaiselle kaudelle paljonko on neljänneksi suurin pistemäärä (15 ensimmäisen pelin pistemääristä)
* laske jokaiselle joukkueelle jokaiselle kaudelle paljonko ero on ollut tähän neljänteen pistemäärään
* suodata mukaan vain ne joukkueet, jotka ko. kaudella ovat loppusijoituksissa 4 parhaan joukossa ja laijittele näistä eroista suurimmat (eniten 4 sijaa jäljessä olleet).

Alkua 1. vaiheesta (15 pelin pistemääristä):  

```
Season       1993-1994  1994-1995  1995-1996  1996-1997  1997-1998  1998-1999
Team                                                                            
Arsenal           26.0       19.0       28.0       31.0       27.0       25.0   
Aston Villa       29.0       13.0       27.0       24.0       18.0       30.0   
Barnsley           NaN        NaN        NaN        NaN       13.0        NaN   
Birmingham         NaN        NaN        NaN        NaN        NaN        NaN   
Blackburn         26.0       33.0       18.0       12.0       30.0        9.0   
```

Vastaus kysymykseen:  
```
 		 	
Season 	     Team          ero_neloseen  loppusija
2001-2002 	  Man United    6 	        3
2012-2013 	  Arsenal       5 	        4
2017-2018 	  Tottenham     4 	        3
2000-2001 	  Leeds 	    4 	        4
2014-2015 	  Arsenal       4 	        3
```

*Ottelusiirtojen yms. vuoksi joukkeet eivät ole pelanneet samaa tahtia, joten "ero 4. sijaan 15 pelin jälkeen" ei ole ollut välttämättä minkään päivän sarjataulukon tilanne (eri joukkueet ovat saaneet 15 peliä pelattua eri aikoina), vaan tässä lasketaan vain jokaiselta joukkueelta sen 15 ensimmäistä ottelua kultakin kaudelta*.

In [644]:
import numpy as np
df = pd.read_csv('https://student.labranet.jamk.fi/~varpe/datananal2k2020/kerta4/epl4.txt')
#df2 = (df['HomePoints'].groupby([df['HomeTeam'], df['Season']]).sum() + df['AwayPoints'].groupby([df['AwayTeam'], df['Season']]).sum()).apply(lambda x: (x[np.isfinite(x)][:15]).sum())
#df = df.stack()
df



,Season,Date,HomeTeam,AwayTeam,HomePoints,AwayPoints
0,1993-1994,1993-08-14,Arsenal,Coventry,0,3
1,1993-1994,1993-08-14,Aston Villa,QPR,3,0
2,1993-1994,1993-08-14,Chelsea,Blackburn,0,3
3,1993-1994,1993-08-14,Liverpool,Sheffield Weds,3,0
4,1993-1994,1993-08-14,Man City,Leeds,1,1
...,...,...,...,...,...,...
10039,2018-2019,2019-05-12,Liverpool,Wolves,3,0
10040,2018-2019,2019-05-12,Man United,Cardiff,0,3
10041,2018-2019,2019-05-12,Southampton,Huddersfield,1,1
10042,2018-2019,2019-05-12,Tottenham,Everton,1,1
